## Imports

In [71]:
#Library used to access and use twitter API
import tweepy  

#Libraries used to process data
import json
import jsonlines
import pandas as pd

## Twitter API info

In [216]:
#Enter your own details from twitter dev account
consumerKey = '*************************'
consumerSecret = '**************************************************'
accessKey = '**************************************************'
accessSecret = '*********************************************'

In [72]:
#Enter your own details from twitter dev account
consumerKey = 'k7tfKcaDGElT23M2rgjI0uP2i'
consumerSecret = 'vI6zFFgfgqYMafORku3J6eZE1OyjalM76RN6b60iO6FwUEiL6o'
accessKey = '1114468605612167169-hA14ZmeetPWissHoxZaW93t4VVx60T'
accessSecret = 'MfvkTr45NAqCx0BqxXm8H2VNTi0zmf7bYj4Fgg7XzuiyT'

## Code to fetch tweets

In [73]:
def fetch_tweets(name):
    '''
    Fetches the tweets from the given handle
    
    Inputs - 
    1) name (string) : The name of the twitter handle you want to fetch tweets from
    Output
    1) tweet_data (list) : The list of tweepy.models.Status objects corresponding to the tweets from the handle

    '''
    
    #Code required to authorise twitter API access
    authorise = tweepy.OAuthHandler(consumer_key=consumerKey,consumer_secret=consumerSecret)
    authorise.set_access_token(accessKey,accessSecret)
    api = tweepy.API(authorise)
    
    #Initialise list to store raw tweets fetched
    tweet_data = []
    
    #Fetch the first 50 tweets from the twitter handle's timeline (or all tweets if there are less than 50)
    data = api.user_timeline(name, count = 50)
    tweet_data.extend(data)
    
    #However, the user may have more than 50 tweets, so we need to go over the remaining tweets
    
    #Find the last id in the first 
    id_last_tweet = data[-1].id - 1
        
    #While the user has tweets remaining, we get tweets in batches of 50
    while(True):
        
        #By specifying max_id = id_last_tweet, we avoid getting duplicate entries/ the first 50 elements again
        data = api.user_timeline(name,count=50, max_id=id_last_tweet)
        tweet_data.extend(data)
        
        #If we have seen all tweets of the user
        if (len(data) == 0):
            break
        
        #Update for next batch
        id_last_tweet = data[-1].id - 1
        


    
    return tweet_data

In [74]:
data = fetch_tweets("midasIIITD")

## Analysing fetched data (Not part of code, just part of how I went through the process

In [75]:
type(data[0])

tweepy.models.Status

In [76]:
data.__len__()

338

In [77]:
data[9]._json

{'created_at': 'Sat Apr 06 17:11:29 +0000 2019',
 'id': 1114576370414292992,
 'id_str': '1114576370414292992',
 'text': 'RT @kdnuggets: Top 8 #Free Must-Read #Books on #DeepLearning #KDN https://t.co/1DtlN91Yjj',
 'truncated': False,
 'entities': {'hashtags': [{'text': 'Free', 'indices': [21, 26]},
   {'text': 'Books', 'indices': [37, 43]},
   {'text': 'DeepLearning', 'indices': [47, 60]},
   {'text': 'KDN', 'indices': [61, 65]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'kdnuggets',
    'name': 'KDnuggets',
    'id': 20167623,
    'id_str': '20167623',
    'indices': [3, 13]}],
  'urls': [{'url': 'https://t.co/1DtlN91Yjj',
    'expanded_url': 'http://ow.ly/2wjT30ohGRL',
    'display_url': 'ow.ly/2wjT30ohGRL',
    'indices': [66, 89]}]},
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name':

In [78]:
data[50]._json["entities"]

{'hashtags': [{'text': 'PortfolioCreationinDesign', 'indices': [39, 65]}],
 'symbols': [],
 'user_mentions': [{'screen_name': 'hcdiiitd',
   'name': 'Human-Centered Design (IIITDelhi)',
   'id': 1090887196754640896,
   'id_str': '1090887196754640896',
   'indices': [3, 12]}],
 'urls': []}

In [79]:
data[82]._json.keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])

In [80]:
data[13]._json["entities"]

{'hashtags': [],
 'symbols': [],
 'user_mentions': [{'screen_name': 'kdnuggets',
   'name': 'KDnuggets',
   'id': 20167623,
   'id_str': '20167623',
   'indices': [3, 13]}],
 'urls': [{'url': 'https://t.co/Z1m0AzlfVv',
   'expanded_url': 'https://buff.ly/2K7XvvN',
   'display_url': 'buff.ly/2K7XvvN',
   'indices': [51, 74]}],
 'media': [{'id': 1114014846817796099,
   'id_str': '1114014846817796099',
   'indices': [75, 98],
   'media_url': 'http://pbs.twimg.com/media/D3XGmdMWAAM7bXv.png',
   'media_url_https': 'https://pbs.twimg.com/media/D3XGmdMWAAM7bXv.png',
   'url': 'https://t.co/ccX4Uhxjn8',
   'display_url': 'pic.twitter.com/ccX4Uhxjn8',
   'expanded_url': 'https://twitter.com/kdnuggets/status/1114014848369799169/photo/1',
   'type': 'photo',
   'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
    'medium': {'w': 640, 'h': 300, 'resize': 'fit'},
    'large': {'w': 640, 'h': 300, 'resize': 'fit'},
    'small': {'w': 640, 'h': 300, 'resize': 'fit'}},
   'source_status_id':

In [81]:
data[9]._json['created_at'].split(" ")[5]

'2019'

In [82]:
json_str = json.dumps(data[9]._json)

json_str

In [ ]:
List of images found

In [ ]:
'''
if 'media' in tweetJSONobj['entities']:
                    media_entities = tweetJSONobj["entities"]["media"]
                    for media_entity in media_entities:
                        print(media_entity['media_url_https'])
'''

https://pbs.twimg.com/media/D3XGmdMWAAM7bXv.png
https://pbs.twimg.com/media/D080rrJXgAAWIQC.jpg
https://pbs.twimg.com/media/D0f6jC9X4AQJsgU.jpg
https://pbs.twimg.com/media/DzToEKZX4AE7rjc.jpg
https://pbs.twimg.com/media/DzQrO_7WsAUu8Na.jpg
https://pbs.twimg.com/media/DyLK6QYX4AIR4ah.jpg
https://pbs.twimg.com/media/DyFhSsFUwAEGU7q.jpg
https://pbs.twimg.com/media/DyDfRcYUYAAl9xP.jpg
https://pbs.twimg.com/media/Dxr-XHqWkAE84QC.jpg
https://pbs.twimg.com/media/DxNp3HBWkAEGVAw.jpg
https://pbs.twimg.com/media/DwzdkXlWwAIvX2J.jpg
https://pbs.twimg.com/media/DsbzH-MWkAUFhZX.jpg
https://pbs.twimg.com/media/DpFT0dZW0AAG5FY.jpg
https://pbs.twimg.com/media/DmBCG2JWwAA5Yu3.jpg
https://pbs.twimg.com/media/DlxuCUiX4AAxzqB.jpg
https://pbs.twimg.com/media/DlRGPMkU8AATQXB.jpg
https://pbs.twimg.com/media/Dk52J-zXoAIn3tm.jpg
https://pbs.twimg.com/media/DksxOnsW0AAOqSL.jpg
https://pbs.twimg.com/media/DkJlHfTW0AA9ZX-.jpg
https://pbs.twimg.com/media/DkDi4wUUYAE-q2o.jpg
https://pbs.twimg.com/media/Dj-a-5oXcAAy

## Code to store data in .jsonl format

In [34]:
def store_tweet_data(tweet_data):
    '''
    Stores the tweets in the .jsonl or jsonlines format 
    Filename: data.jsonl
    
    Inputs 
    1) tweet_data (list) : The list of tweepy.models.Status objects corresponding to the tweets from the handle   
    '''
    
    #Convert the Status objects data to strings that are in json form
    json_data = [json.dumps(status._json) for status in tweet_data]
    
    # Store the data in the data.jsonl file
    with open('data.jsonl', 'ab') as f:  
        with jsonlines.Writer(f) as writer:
            writer.write(json_data)

In [35]:
store_tweet_data(data)

In [85]:
def extract_tweet_info(file):
    '''
    Extracts the required info from the give .jsonl file
    
    Inputs 
    1) file (str) : The filename of the .jsonl file
    Outputs
    1) data_df (pandas.Dataframe): The pandas Dataframe contained the info
    '''
    
    #Initialize lists for the info
    texts = []
    dates = []
    months = []
    times = []
    years = []
    n_favorites = []
    n_retweets = []
    n_images = []

    #Open file and run through the tweets' json files
    with jsonlines.open(file) as reader:
        for tweets in reader:
            for tweet in tweets:
    
                #Retrieve json in dict form from tweet
                tweetJSONobj = json.loads(tweet)
            
                #Collect the time,date,month,year info  
                '''
                Example of how tweepy stores this "created at" data : 'Wed Apr 03 18:31:53 +0000 2019'
                '''
                created_at = tweetJSONobj["created_at"]
                times.append(created_at.split(" ")[3])
                dates.append(created_at.split(" ")[2])
                months.append(created_at.split(" ")[1])
                years.append(created_at.split(" ")[5])
                
                
                #Collect the text of the tweet
                text_str = tweetJSONobj["text"]
                texts.append(text_str)
                
                #Collect number of favorites and retweets
                n_favorites.append(tweetJSONobj["favorite_count"])
                n_retweets.append(tweetJSONobj["retweet_count"])
                
                #Find the number of images  
                image_count = 0
                if 'media' in tweetJSONobj['entities']:
                    media_entities = tweetJSONobj["entities"]["media"]
                    for media_entity in media_entities:
                        if(media_entity['media_url_https'].split(".")[-1] == 'jpg' \
                           or media_entity['media_url_https'].split(".")[-1] == 'png'):
                            image_count = image_count + 1

              

                
                if(image_count == 0):
                    n_images.append(None)
                else:
                    n_images.append(image_count)

                
                    
                    
        #Store the extracted info in a Dataframe            
        data_df = pd.DataFrame()
        data_df['Text']  = texts
        data_df["Time"] = times
        data_df["Date"] = dates
        data_df["Month"] = months
        data_df["Year"] = years
        data_df["No. Favorites"] = n_favorites
        data_df["No. Retweets"] = n_retweets
        data_df["No. Images"] = n_images
        
        return(data_df)

                
    

In [86]:
data_extracted.head(100)

,Text,Time,Date,Month,Year,No. Favorites,No. Retweets,No. Images
0,We request all students whose interview are sc...,11:43:24,07,Apr,2019,0,1,None
1,"Other queries: ""none of the Tweeter Apis give ...",06:55:19,07,Apr,2019,3,2,None
2,"Other queries: ""do we have to make two differe...",06:53:38,07,Apr,2019,4,1,None
3,"Other queries: ""If using Twitter api, it does ...",05:32:27,07,Apr,2019,4,1,None
4,Response to some queries asked by students on ...,05:29:40,07,Apr,2019,6,1,None
5,RT @kdnuggets: Top 8 #Free Must-Read #Books on...,17:11:29,06,Apr,2019,0,2,None
6,@nupur_baghel @PennDATS Congratulation @nupur_...,16:43:27,06,Apr,2019,15,3,None
7,We have emailed the task details to all candid...,16:08:37,05,Apr,2019,10,1,None
8,RT @rfpvjr: Our NAACL paper on polarization in...,04:05:11,05,Apr,2019,0,16,None
9,RT @kdnuggets: Effective Transfer Learning For...,04:04:43,05,Apr,2019,0,10,None


# References 

Tweepy docs : https://buildmedia.readthedocs.org/media/pdf/tweepy/v3.2.0/tweepy.pdf

json docs : https://docs.python.org/2/library/json.html

Status object structure : https://gist.github.com/dev-techmoe/ef676cdd03ac47ac503e856282077bf2

StackOverflow -
Convert Tweepy Status object into JSON : https://stackoverflow.com/questions/27900451/convert-tweepy-status-object-into-json